In [1]:
import pandas as pd
import numpy as np
import toad
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from xgboost import XGBClassifier as XGB
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc
import warnings
warnings.filterwarnings("ignore")



In [2]:
target = 'target'
to_drop = []
train_date = ['202210', '202211', '202212']
oot_date = ['202301']


data = pd.read_csv('/Users/apple/Desktop/target2.csv')
data['target'] = data['target'].replace({'good': 0, 'bad': 1})
data = data.drop(columns = to_drop)

data['fill_bill_date'] = data['fill_bill_date'].astype(str)
data['dep_prov_no'] = data['dep_prov_no'].astype(str)
data['usr_job'] = data['usr_job'].astype(str)
data['schooling'] = data['schooling'].astype(str)
data = data[data['dep_prov_no'] == '22']

columns = list(data.columns)
columns[:-1] = ['X_' + str(i+1) for i in range(len(data.columns)-1)]
data.columns = columns


train_data = data[data['X_1'].isin(train_date)]
oot_data = data[data['X_1'].isin(oot_date)]
train_data = train_data.drop(columns = ['X_1','X_4'])
oot_data = oot_data.drop(columns = ['X_1','X_4'])

# 将数据分割成训练集的比例
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(train_data, test_size=0.2, random_state=55,stratify=train_data[target])
# train_data.shape,oot_data.shape
# toad.detect(train_set)


In [6]:

train_selected, dropped = toad.selection.drop_empty(train_set,threshold = 0.8, return_drop=True)
print('删除缺失值多的变量-后',train_selected.shape)
train_selected.head(3)
# 
# train_selected, dropped = toad.selection.drop_iv(train_selected,threshold = 0.1,return_drop=True)
# print('删除低iv的变量-后',train_selected.shape)
# 
# train_selected, dropped = toad.selection.drop_corr(train_selected,target = target,threshold = 0.7, return_drop=True)
# print('删除相关性高的变量-后',train_selected.shape)
# 训练集

train_set_X = train_selected.drop(columns=target)
train_set_Y = train_selected[target]

test_set_X = test_set[train_set_X.columns]
test_set_Y = test_set[target]

oot_set_X = oot_data[train_set_X.columns]
oot_set_Y = oot_data[target]

删除缺失值多的变量-后 (25711, 148)


In [10]:
# one-hot 转换

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), make_column_selector(dtype_include=object))
    ],
    remainder='passthrough'
)
processed_train_data = preprocessor.fit_transform(train_set_X)
processed_test_data = preprocessor.fit_transform(test_set_X)
processed_oot_data = preprocessor.fit_transform(oot_set_X)

print(processed_train_data.shape, processed_test_data.shape, processed_oot_data.shape)
# 获取转换后的特征名称
columns = preprocessor.get_feature_names_out()
# 创建新的DataFrame，使用新的列名
onehot_train_X = pd.DataFrame(processed_train_data, columns=columns)
onehot_test_X = pd.DataFrame(processed_train_data, columns=columns)
onehot_oot_X = pd.DataFrame(processed_train_data, columns=columns)
print(onehot_train_X.shape, onehot_test_X.shape, onehot_oot_X.shape)


(25711, 174) (6428, 174) (9361, 174)
(25711, 174) (25711, 174) (25711, 174)


In [4]:

encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
# 拟合编码器到训练数据
encoder.fit(train_set_X)

# 转换训练数据和测试数据
onehot_train_X = encoder.transform(train_set_X)
onehot_test_X = encoder.transform(test_set_X)
onehot_oot_X = encoder.transform(oot_set_X)

# onehot_train_X.shape,onehot_test_X.shape,onehot_oot_X.shape
# 获取列名
columns = encoder.get_feature_names_out(input_features=train_set_X.columns)

# 转换为DataFrame并设置列名
onehot_train_X = pd.DataFrame(onehot_train_X, columns=columns)
onehot_test_X = pd.DataFrame(onehot_test_X, columns=columns)
onehot_oot_X = pd.DataFrame(onehot_oot_X, columns=columns)
print(len(columns))
# columns
# onehot_train_X_df.head(10)
onehot_train_X.shape,onehot_test_X.shape,onehot_oot_X.shape

删除缺失值多的变量-后 (25711, 148)
8632


((25711, 8632), (6428, 8632), (9361, 8632))

In [16]:
xgb = XGB(
    colsample_bytree= 0.8,
    learning_rate= 0.1,
    max_depth= 4,
    n_estimators= 100,
    subsample= 0.8,
    seed=11
)

xgb.fit(onehot_train_X,train_set_Y)

xgb_predict_proba = xgb.predict_proba(onehot_train_X)

xgb_def_rate = xgb_predict_proba[:,1]

Exception ignored in: <function Booster.__del__ at 0x14eb1a840>
Traceback (most recent call last):
  File "/Users/apple/anaconda3/lib/python3.11/site-packages/xgboost/core.py", line 1651, in __del__
    def __del__(self) -> None:

KeyboardInterrupt: 
Exception ignored in: <function DMatrix.__del__ at 0x14eb18b80>
Traceback (most recent call last):
  File "/Users/apple/anaconda3/lib/python3.11/site-packages/xgboost/core.py", line 797, in __del__
    _check_call(_LIB.XGDMatrixFree(self.handle))
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
#计算AUC
fpr, tpr, thresh_lr = roc_curve(train_set_Y,xgb_def_rate)
# 计算 AUC
roc_auc = auc(fpr, tpr)
# 计算 KS 统计量
ks_statistic = np.max(tpr - fpr)
print(f"KS Statistic on Train Set: {ks_statistic}")
print(f"AUC on Test Set: {roc_auc}")

# 测试-- begin
# 使用训练好的模型进行预测
test_predict_proba = xgb.predict_proba(onehot_test_X)[:, 1]
# 真实标签
true_labels = test_set_Y
# 计算 ROC 曲线的各个点
fpr, tpr, thresholds = roc_curve(true_labels, test_predict_proba)
# 计算 KS 统计量
ks_statistic = np.max(tpr - fpr)
# 计算 AUC
roc_auc = auc(fpr, tpr)

print(f"KS Statistic on Test Set: {ks_statistic}")
print(f"AUC on Test Set: {roc_auc}")
# 测试-- end

# oot-- begin
# 使用训练好的模型进行预测
oot_predict_proba = xgb.predict_proba(onehot_oot_X)[:, 1]
# 真实标签
true_labels = oot_set_Y
# 计算 ROC 曲线的各个点
fpr, tpr, thresholds = roc_curve(true_labels, oot_predict_proba)
# 计算 KS 统计量
ks_statistic = np.max(tpr - fpr)
# 计算 AUC
roc_auc = auc(fpr, tpr)

print(f"KS Statistic on OOT Set: {ks_statistic}")
print(f"AUC on OOT Set: {roc_auc}")
# 测试-- end
# print(toad.metrics.PSI(pred_train[:,1],pred_test))

In [33]:
print(toad.metrics.PSI(xgb_def_rate,test_predict_proba))
print(toad.metrics.PSI(xgb_def_rate,oot_predict_proba))
print('--------')

0.14727406984061786
0.13892559495234413
--------


In [ ]:

# print(calculate_psi(xgb_def_rate,test_predict_proba))
# print(calculate_psi(xgb_def_rate,oot_predict_proba))
buckets = 10
expected_array = xgb_def_rate
actual_array = oot_predict_proba
# 将分数切割成分位数分箱
breakpoints = np.linspace(0, 100, buckets + 1)
breakpoints = np.percentile(expected_array, breakpoints)
# 确保分箱边界的唯一性
breakpoints = np.unique(breakpoints)
print(breakpoints)

# 使用分位数计算分箱后每个箱子的实际和预期人数
expected_counts = np.histogram(expected_array, breakpoints)[0]
actual_counts = np.histogram(actual_array, breakpoints)[0]

# 计算每个箱子的占比
expected_probs = expected_counts / expected_counts.sum()
actual_probs = actual_counts / actual_counts.sum()

# 处理0概率的情况，以避免0概率导致的计算错误
actual_probs += np.finfo(float).eps
expected_probs += np.finfo(float).eps

# 计算PSI
psi_values = (actual_probs - expected_probs) * np.log(actual_probs / expected_probs)
psi = np.sum(psi_values)
psi

# oot_predict_proba

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# 假设xgb是已经训练好的XGBoost模型
# train_set_X是用于训练模型的特征数据集

# 获取特征重要性并创建一个DataFrame
feature_importances = pd.DataFrame(xgb.feature_importances_,
                                   index=train_set_X.columns,
                                   columns=['importance'])

# 对特征重要性进行降序排序
sorted_features = feature_importances.sort_values(by='importance', ascending=False)

# 选择前30个最重要的特征
top_30_features = sorted_features.head(30)

# 打印结果
top_30_features

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))


# 绘制ROC曲线
plt.subplot(1, 2, 1)
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.fill_between(fpr, fpr, tpr, color='grey', alpha=0.3, label='Area under curve')
plt.plot([fpr[ks_statistic]], [tpr[ks_statistic]], marker='o', markersize=5, color="red")
plt.text(fpr[ks_statistic], tpr[ks_statistic], f'KS={ks_statistic:.2f}', fontsize=12, ha='right')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")

# 绘制KS曲线
plt.subplot(1, 2, 2)
plt.plot(thresholds, tpr, label='TPR', color='blue')
plt.plot(thresholds, fpr, label='FPR', color='green')
plt.plot([thresholds[ks_statistic], thresholds[ks_statistic]], [fpr[ks_statistic], tpr[ks_statistic]], 'k--', color="red")
plt.fill_between(thresholds, fpr, tpr, where=tpr>fpr, color='grey', alpha=0.3)
plt.gca().invert_xaxis()  # 阈值由高到低
plt.xlabel('Thresholds')
plt.ylabel('Rate')
plt.title('KS Curve')
plt.legend(loc="upper right")

plt.tight_layout()
plt.show()
